In [133]:
import numpy as np
import os, sys, datetime, string
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/MEPS/')

import matplotlib.pyplot as plt
from scipy import stats
from mpl_toolkits.basemap import Basemap
import netCDF4
import matplotlib as mpl
import save_fig as sF
import createFolder as cF
from calc_station_properties import find_station_yx

In [134]:
import matplotlib.colors as colors

import gdal
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cbook import get_sample_data
import pandas as pd
from matplotlib.colors import LightSource
from pyproj import Proj, transform

In [135]:
savefig = 0   # 1 = yes, 0 = no
fig_dir = '../../Figures/MEPS/area_precip'
cF.createFolder(fig_dir)
form = 'png'

In [136]:
def Lambert_map(lllon, lllat, urlon, urlat, lat0, lon0, res='l', fill=False, zoom=False):
    """lllon - lon lower left corner ...
    lat0 - latitude standard parallel, should be somewhere in the center of the domain
    lon0 - the parallel longitude
    lllon - lower left longitude ...
    http://matplotlib.org/basemap/api/basemap_api.html"""
    rsphere=(6378137.00,6356752.3142)
    map = Basemap(llcrnrlon=lllon, llcrnrlat=lllat, urcrnrlon=urlon, urcrnrlat=urlat, rsphere=rsphere,
            resolution=res,area_thresh=1000.,projection='lcc', lat_0=lat0,lon_0=lon0)
#    map = Basemap(llcrnrlon=lllon, llcrnrlat=lllat, urcrnrlon=urlon, urcrnrlat=urlat, rsphere=rsphere,
 #           resolution=res,area_thresh=1000.,projection='lcc', lat_1=lllon,lon_0=lon0)
    # Draw the latitudes and the longitudes
#    parallels = np.arange(0.,90,5.)
 #   map.drawparallels(parallels,labels=[True,False,False,False],fontsize=16)    
  #  meridians = np.arange(10.,361.,10.)
   # map.drawmeridians(meridians,labels=[False,False,False,True],fontsize=16)
    if zoom == False:
        map.drawmeridians(np.arange(0,90,10),labels=[0,0,0,1],fontsize=16)
        map.drawparallels(np.arange(10,361,4),labels=[1,0,0,0],fontsize=16)
    else:
#        map.drawmeridians(np.arange(0,90,3),labels=[0,0,0,1],fontsize=16)
 #       map.drawparallels(np.arange(0,361,2),labels=[1,0,0,0],fontsize=16)
        map.drawmeridians(np.arange(0,90,0.1),labels=[0,0,0,1],fontsize=16)
        map.drawparallels(np.arange(0,361,0.05),labels=[1,0,0,0],fontsize=16)

    # Draw the coastline
    map.drawcoastlines()#color='0.5')
    
    

    if fill:
        map.drawlsmask(#land_color='0.8', 
            ocean_color='gainsboro')
    
#    if zoom == False:
    
    ### plot MEPS area
 #       for i in range(0,lato.shape[0],12):
  #          xs, ys = map(lono[i], lato[i])
   #         map.plot(xs,ys, color = 'orange', marker = 'o', markersize = 10, linestyle = '-', linewidth = 10)
    #    for i in range(0,lato2.shape[0],12):
     #       xs2, ys2 = map(lono2[i], lato2[i])
      #      map.plot(xs2,ys2, color = 'orange', marker = 'o', markersize = 10, linestyle = '-', linewidth = 10)

       # xs, ys = map(lono[739], lato[739])
        #map.plot(xs,ys, color = 'orange', marker ='o', markersize = 10, linestyle = '-', linewidth = 10, label = 'MEPS domain')
        #lgd = plt.legend(loc='lower left',fontsize=18)

    #map.drawmapboundary(fill_color='gainsboro')
        
    return map    


In [137]:
def PlotContours(Lon, Lat, psurf, map, nrlevels=10, leveldist=None,levels=None, numbers=True, color= 'k'):
    """ contours for example the pressure
    nrlevels - gives the number of displayed levels
    leveldist - gives distance between levels, if specified the nlevels is ignored
    levels - can be an array that specifies the levels to display, if specified nrlevels and leveldist are ignored
    numbers - True if the contours are labeled
    color - color of the contours (None is s color map)"""
    if levels is not None:
        cs= map.contour(Lon, Lat, psurf, levels, linewidths= 1. , colors= color)
    elif leveldist is not None:
        levels= np.arange(np.round(np.min(psurf)- np.min(psurf)%leveldist), np.round(np.max(psurf)+ leveldist), leveldist)
        cs= map.contour(Lon, Lat, psurf, levels, linewidths= 1. , colors= color)        
    else:
        cs= map.contour(Lon, Lat, psurf, nrlevels, linewidths= 1. , colors= color)#, colors= 6*['b']+ 6*['r'],)
    if numbers == True: plt.clabel(cs, fontsize=10, inline=1, fmt='%1.0f', color= 'black')
    #plt.tight_layout()

In [138]:
def PlotColorMap4(Lon, Lat, data, map, maxlevel= None, symetric=True, bounds=None, label='', color= 'RdBu', boxnr= 21):
    """ plot a color map, e.g. vertical velocity
    if symetric == True it is symetric around 0 and the maxlevel is calculated automatically
    best version of PlotColorMap"""
    
    if color== 'RdBu': colors= [(plt.cm.RdBu_r(h)) for h in range(256)]
    elif color== 'seismic': colors= [(plt.cm.seismic(h)) for h in range(256)]
    elif color== 'blue': colors= [(plt.cm.Blues(h)) for h in range(256)]
    elif color== 'inverse_blue': colors= [(plt.cm.Blues(h)) for h in range(255, 0, -1)]

    elif color == 'red': colors= ['azure']+[(plt.cm.Reds(h)) for h in range(256)]
    else: print('wrong color')
#    if bounds != None: boxnr = len(bounds)
    new_map = plt.matplotlib.colors.LinearSegmentedColormap.from_list('new_map', colors) #, N=boxnr)

    if bounds is None:
        if maxlevel is not None: minlevel= maxlevel
        if maxlevel is None and bounds is None:
            if symetric is True:
                maxlevel, minlevel= np.max(np.abs(data)), -np.max(np.abs(data))
            else:
                maxlevel, minlevel= np.max(data), np.min(data)       
        
        bounds= np.round(np.linspace(minlevel, maxlevel, boxnr+1), int(np.log10(85/maxlevel)))
#        bounds= np.round(list(np.linspace(-maxlevel, 0, boxnr//2+1))+list(np.linspace(0, maxlevel, boxnr//2+1)), int(np.log10(85/maxlevel)))        
#        print(maxlevel)

    Lon= 0.5* (Lon[1:, 1:]+ Lon[:-1, :-1])
    Lat= 0.5* (Lat[1:, 1:]+ Lat[:-1, :-1])
        
    norm= mpl.colors.BoundaryNorm(bounds, new_map.N)
    cs= map.pcolormesh(Lon, Lat, data[1:, 1:], norm= norm, cmap=new_map, alpha= 1.)
    cb = map.colorbar(cs, boundaries= bounds, norm= norm, location='right',pad='3%',extend='max')
        
    cb.set_label(label, size=18)    
    cb.ax.tick_params(labelsize=16)


In [139]:
champ = 255.

no22 = np.array([80,80,81])/champ

In [140]:
time = 9
year = 2016
month = 12
date = 26

In [141]:
#url = ('http://thredds.met.no/thredds/dodsC/meps25epsarchive/2016/12/24/meps_mbr0_pp_2_5km_20161224T12Z.nc')
url = ('http://thredds.met.no/thredds/dodsC/meps25epsarchive/%s/%s/%s/meps_mbr0_pp_2_5km_%s%s%sT00Z.nc' 
       %(year,month,date,year,month,date))
#url = ('http://thredds.met.no/thredds/dodsC/meps25epsarchive/2016/12/21/meps_mbr0_pp_2_5km_20161221T00Z.nc')
dataset = netCDF4.Dataset(url)

land= dataset.variables['land_area_fraction'][:]
lonpp= dataset.variables['longitude'][:]
latpp= dataset.variables['latitude'][:]

x_wind = dataset.variables['x_wind_10m'][time,:,:]
y_wind = dataset.variables['y_wind_10m'][time,:,:]

#T_2m= dataset.variables['air_temperature_2m'][:]
alti= dataset.variables['altitude'][:]

precipitation_amount_acc = dataset.variables['precipitation_amount_acc'][time,0,:,:]
precipitation_amount = dataset.variables['precipitation_amount'][time,:,:]


#dataset.close()

In [142]:
dataset

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format DAP2):
    Conventions: CF-1.0
    institution: Norwegian Meteorological Institute, met.no
    creator_url: met.no
    summary: MEPS (MetCoOp-Ensemble Prediction System) is a convection-permitting atmosphere ensemble model covering Scandinavia and the Nordic Seas. It has horizontal resolution of 2.5 km, has 65 vertical levels and 10 members. MEPS is ran four times daily (00,06,12,18) with three-hourly cycling for data assimilation. Member 0 and 1 are ran up to 66 hours, the rest up to 48 hours. Boundary data is from ECMWF, and initial perturbations are based on the SLAF method. Model code based on HARMONIE cy40h1.1
    title: unknown
    geospatial_lat_min: 51.0
    geospatial_lat_max: 88.0
    geospatial_lon_min: -20.0
    geospatial_lon_max: 80.0
    references: unknown
    license: http://met.no/sb5-met.no/Forskning/Informasjonsteknologi/Tilgang_til_data/
    comment: none
    history: Tue Oct 18 

In [143]:
grid_x, grid_y = find_station_yx(latpp, lonpp, 59+48.73/60, 7+12.87/60)

In [144]:
alti[grid_y[0],grid_x[0]]

1041.2291

In [145]:
lonpp[grid_y[0],grid_x[0]]

7.2159269574384544

In [146]:
latpp[grid_y[0],grid_x[0]]

59.80292042731746

In [147]:
#### Station map
#plt.figure(1)
fig = plt.figure(figsize=(9,8))
plt.clf()
map= Lambert_map(lllon=7.05, lllat=59.73, urlon=7.35, urlat=59.91, lat0= 63.5, lon0= 15, res='i', fill=False,zoom=True)

Lonpp,Latpp = map(lonpp,latpp)

 
#levels = [850, 900,950,1000,1050,1100,1150, 1200, 1250, 1300,1350,1400]
levels = np.arange(700,1800,50)
PlotContours(Lonpp, Latpp, alti, map, leveldist=None,levels=levels, numbers=True, color= 'gray')

#cmap = colors.ListedColormap([no0, no1, no2, no3, no4, no5, no6, no7, no8, no9, no10, \
 #                             no11, no12, no13, no14, no15, no16, no17, no18, no19, no20, \
  #                           no21])
#norm = colors.BoundaryNorm(boundaries = levels, ncolors=cmap.N)

#boxnr = 9
boxnr = 20
colors = [(plt.cm.Blues(h)) for h in range(256)]
new_map = plt.matplotlib.colors.LinearSegmentedColormap.from_list('new_map', colors,  N=boxnr)
#norm = colors.BoundaryNorm(boundaries = levels2, ncolors=cmap.N)
cs = map.pcolormesh(Lonpp, Latpp, precipitation_amount, cmap=new_map,alpha=1.,
                   vmin = 0., vmax =10)
cb = plt.colorbar(cs,# boundaries= levels, 
                 # location='right',
                  extend='max')
cb.set_ticks(np.arange(boxnr+1))
    



cb.set_label('Precipitation amount [mm]', size=18)    
cb.ax.tick_params(labelsize=16)
plt.gca().set_aspect('equal', adjustable='box')

### plot wind barbs
map.barbs(Lonpp,Latpp,x_wind,y_wind,barbcolor = [no22], pivot = 'middle')
        


#cb.ax.set_xticklabels([700, '', '', '', 900,'','','',1100,'','','',1300,'','','',1500,
 #                       '','','',1700,''])  # horizontal colorbar

namestat = ['Haukeliseter']#,'Model']#, 'lower left', 'upper right']#,'grid point']
lonstat = [7+12.87/60]#,lonpp[grid_y[0],grid_x[0]]]#,7.05, 7.4]#,7.2]
latstat= [59+48.73/60]#,latpp[grid_y[0],grid_x[0]]]#,59.65, 59.9]#,59.8]
xpt, ypt= map(lonstat, latstat)
map.plot(xpt,ypt,color='k', marker='X',markersize=12)

namestat2 = ['Model']#, 'lower left', 'upper right']#,'grid point']
lonstat2 = [lonpp[grid_y[0],grid_x[0]]]#,7.05, 7.4]#,7.2]
latstat2= [latpp[grid_y[0],grid_x[0]]]#,59.65, 59.9]#,59.8]

xpt2, ypt2= map(lonstat2, latstat2)
map.plot(xpt2,ypt2,'ko')

for i in range(len(namestat)):
    plt.text(xpt2[i], ypt2[i], namestat2[i], fontsize=18,fontweight='bold', ha='center',va='bottom',color='black')

fig_name = '%s%s%s_%s.png' %(year,month,date,time)

if savefig == 1:
    sF.save_figure_portrait(fig_dir,fig_name,form)
    print('saved: %s/%s' %(fig_dir,fig_name))
else: 
    plt.show()
plt.close()

saved: ../../Figures/MEPS/area_precip/20161226_9.png
